<a href="https://colab.research.google.com/github/blue0620/NDLOCR-GoogleColabVersion/blob/main/NDLOCR_googlecolabversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. GPUの情報を確認する

In [2]:
!nvidia-smi

Tue Apr 26 11:46:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    44W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 1. NDLOCRのリポジトリをcloneする(--recursiveを忘れずに！)

In [ ]:
!git clone --recursive https://github.com/ndl-lab/ndlocr_cli

# 2. 必要なパッケージをインストールする

In [35]:
!pip install -r {PROJECT_DIR}/requirements.txt
!pip install torch==1.8.1+cu112 torchvision==0.9.1+cu111 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!pip install mmcv-full==1.4.0 -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.8.0/index.html
##numpyのバージョン問題でcolabでは動かなかったのでアップデートする(参考:https://stackoverflow.com/questions/66060487/valueerror-numpy-ndarray-size-changed-may-indicate-binary-incompatibility-exp)
!pip install --upgrade numpy

     |████████████████████████████████| 312 kB 5.1 MB/s 
     |████████████████████████████████| 299 kB 69.3 MB/s 
     |████████████████████████████████| 1.5 MB 84.5 MB/s 
     |████████████████████████████████| 14.8 MB 69.2 MB/s 
     |████████████████████████████████| 50.4 MB 1.3 MB/s 
     |████████████████████████████████| 26.5 MB 36.0 MB/s 
     |████████████████████████████████| 31.2 MB 26.7 MB/s 
     |████████████████████████████████| 3.1 MB 55.6 MB/s 
     |████████████████████████████████| 109.3 MB 127.5 MB/s 
     |████████████████████████████████| 488 kB 85.1 MB/s 
     |████████████████████████████████| 50 kB 8.0 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0


In [9]:
PROJECT_DIR="/content/ndlocr_cli"

In [ ]:
%cd {PROJECT_DIR}/src/ndl_layout/mmdetection
!python setup.py bdist_wheel
!pip install dist/*.whl
%cd /content

# 4. OCRに必要な学習済みモデルをダウンロードする

In [5]:
%cd {PROJECT_DIR}
!wget https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/mojilist_NDL.txt -P ./src/text_recognition/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/ndlenfixed64-mj0-synth1.pth -P ./src/text_recognition/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/ndl_layout/ndl_layout_config.py -P ./src/ndl_layout/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/ndl_layout/epoch_140_all_eql_bt.pth -P ./src/ndl_layout/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/separate_pages_ssd/weights.hdf5 -P ./src/separate_pages_ssd/ssd_tools
%cd /content/

/content/ndlocr_cli
--2022-04-26 11:50:56--  https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/mojilist_NDL.txt
Resolving lab.ndl.go.jp (lab.ndl.go.jp)... 108.156.83.12, 108.156.83.89, 108.156.83.45, ...
Connecting to lab.ndl.go.jp (lab.ndl.go.jp)|108.156.83.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28088 (27K) [text/plain]
Saving to: ‘./src/text_recognition/models/mojilist_NDL.txt’

mojilist_NDL.txt    100%[===================>]  27.43K  --.-KB/s    in 0.004s  

2022-04-26 11:50:56 (6.54 MB/s) - ‘./src/text_recognition/models/mojilist_NDL.txt’ saved [28088/28088]

--2022-04-26 11:50:56--  https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/ndlenfixed64-mj0-synth1.pth
Resolving lab.ndl.go.jp (lab.ndl.go.jp)... 108.156.83.12, 108.156.83.89, 108.156.83.45, ...
Connecting to lab.ndl.go.jp (lab.ndl.go.jp)|108.156.83.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191768312 (183M) [application/x-www-form-urlencoded]
Savin

# 5. 環境変数を追加する

In [30]:
import os
os.environ["PYTHONPATH"]=os.environ["PYTHONPATH"]+":"+f"{PROJECT_DIR}/src/text_recognition/deep-text-recognition-benchmark"

# 6. PDFを画像に変換するためのパッケージのインストール

In [47]:
!apt-get install poppler-utils
!pip install pdf2image 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 40 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.12 [154 kB]
Fetched 154 kB in 0s (810 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 155501 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.62.0-2ubuntu2.12_amd64.deb ...
Unpacking poppler-utils (0.62.0-2ubuntu2.12) ...
Setting up poppler-utils (0.62.0-2ubuntu2.12) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
  Using cached pdf2image-1.16.0-py3-none-any.whl (10 kB)


# 7. テキスト化したいPDFをダウンロードする
今回は、ROIS-DS人文学オープンデータ共同利用センター(http://codh.rois.ac.jp/
)が提供している

近代雑誌データセット　http://codh.rois.ac.jp/modern-magazine/
から、

東洋学芸雑誌(https://dglb01.ninjal.ac.jp/ninjaldl/bunken.php?title=toyogakuge)

第一号(https://dglb01.ninjal.ac.jp/ninjaldl/toyogakuge/001/PDF/tygz-001.pdf)

をダウンロードしてみます。

In [57]:
!curl https://dglb01.ninjal.ac.jp/ninjaldl/toyogakuge/001/PDF/tygz-001.pdf -o /content/tygz-001.pdf

/content
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14.1M  100 14.1M    0     0  5517k      0  0:00:02  0:00:02 --:--:-- 5517k


# 8. PDFをjpeg画像に変換する

In [64]:
from pathlib import Path
from pdf2image import convert_from_path
import os
pdf_path = Path("/content/tygz-001.pdf")
os.makedirs("/content/tygz-001/img",exist_ok=True)
img_path=Path("/content/tygz-001/img")

convert_from_path(pdf_path, output_folder=img_path,fmt='jpeg',output_file=pdf_path.stem,dpi=100)

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7F679E4F8810>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7F679E4F8150>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7F679E4F87D0>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7F679E4F8850>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7F679E4F8A90>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7F679E4F8E10>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7F679E3E5050>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7F679E3E5150>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7F679E3E5250>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7F679E4F8090>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7F679E3E5410>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB siz

# 9. OCRの実行

/content/tygz-001以下のimgディレクトリ内の画像を処理し、
/content/tygz-001_outputに出力する場合

In [68]:
%cd {PROJECT_DIR}
!python main.py infer /content/tygz-001 /content/tygz-001_output -s s -x

/content/ndlocr_cli
start inference !
input_root : /content/tygz-001
output_root : /content/tygz-001_output
config_file : config.yml
Using TensorFlow backend.
load from config=src/ndl_layout/models/ndl_layout_config.py, checkpoint=src/ndl_layout/models/epoch_140_all_eql_bt.pth
set up EQL (version NDL), 9 classes included.
load checkpoint from local path: src/ndl_layout/models/epoch_140_all_eql_bt.pth
No Transformation module specified
No SequenceModeling module specified
model input parameters 32 1200 20 1 512 256 7085 100 None ResNet None CTC
loading pretrained model from src/text_recognition/models/ndlenfixed64-mj0-synth1.pth
[{'input_dir': '/content/tygz-001', 'img_list': ['/content/tygz-001/img/tygz-0010001-01.jpg', '/content/tygz-001/img/tygz-0010001-02.jpg', '/content/tygz-001/img/tygz-0010001-03.jpg', '/content/tygz-001/img/tygz-0010001-04.jpg', '/content/tygz-001/img/tygz-0010001-05.jpg', '/content/tygz-001/img/tygz-0010001-06.jpg', '/content/tygz-001/img/tygz-0010001-07.jpg', 

# 10. 結果の確認

In [75]:
import glob
import os
for fpath in sorted(glob.glob("/content/tygz-001_output/tygz-001/txt/*_main.txt")):
    with open(fpath) as f:
        txtdata=f.read()
        print(os.path.basename(fpath).replace("_main.txt",""))
        print(txtdata)

tygz-0010001-01
明治十六年九月再版
東法學藝羅誌


tygz-0010001-02



tygz-0010001-03
ホ或ハ此雜誌ノ讀三難キヲ困シムモノ
上二渉レル平易ナル文章ヲモ其間ニ雜
ヘ甘苦相半ナラシメ以テ世人ノ望二負
ク無キヲ期スト云爾
ナキ二非ス因テ更二其區域ヲ廣メ文藝
トノ三ヲ討論スル二非スト雖トモ世尚

常ニ憂フルトコロナリ故ニ之ヲ救ハン
我邦人ノ理學ノ思想二乏シキハ識者ノ
カ爲ニ此雜誌二理學二關係アル文章ヲ
掲載シテ其性質及ヒ功用ヲ世ニ明ニヤ
緒言


tygz-0010001-04



tygz-0010001-05
○理學ノ快樂一二二、
○平賀源内傳片山沖堂一一五、
○非時事小言論上田秀成
○答東京經濟雜誌井上哲次郎一一〇、
○ハムレット(反譯)尚今居士一一七、
○理學ノ快樂
○遊日光山雜記谷田部梅吉報三八、六四、
○水ヲ燃料ニ供ス
○度量衡説附貨幣鮫島晋
失矢田部良吉一二七、一五
上田秀成四七、
一三六
三三、八四
一五八、
○本誌改良ノ因由
皇帝陛下ノ万歳
ヲ述ヘテ
○羅馬字ヲ以テ日
一〇六、一四二
本語ヲ綴ルノ説
ヲ祝シ奉ル
〇一、一三六

加藤弘之一、一六、
論説之部
論説人名丁數
○世界開闢〓論隈本有尚五六、
○分子ノ解磯野徳三郎七二、
○西哲叢談シヨン、ダルトン六五、
〇)〓
○西哲叢談ジヨン、ダルトン
○ヒューム氏自由
〓
井上哲二郎五三
シテハ如何ヲ論九一
○以ヲ〓子ヲ井上哲二郎五三、
テ人才ヲ得ルノ
四九、七一、
○人爲淘汰ニヨリ
術ヲ論ス
九一
○題有機化學沿革
評スルヲ評ス
○泰西人ノ孔子ヲ
○ヒューム氏自由
史後


tygz-0010001-06
樫村清徳二一二、
○文章論小中村清矩二〇九、
○理學功用辨杉浦重剛二一一
○支那紙幣史畧平沼淑郎一六〇、一八八
尚今居士二二三、
○職業教育論手島精一二一四、
○人類ノ紀元松下丈吉一五五、一七九
○書ハ美術ナラス小山正太郎
○抜刀隊ノ歌外山正一一七〇
○岩垣月洲傳杉浦正臣一六九
鈎玄堂主人一六七
○讀原道評杉浦正臣二〇三、
○謠曲ハ肺氣腫症
之ヲ廢棄セン歟
二二七、
ヲ評ス
一七二、二〇五、
○カムプベル氏英
ノ一因タリ抑モ
定スル論
國海軍の詩
○クック氏ノ演説
○謠曲ハ肺氣腫症
之ヲ廢棄セン歟

○讀韓氏原道井上哲次郎一三九、
中川元一八一
井上哲二